In [34]:
import boto3

In [38]:
client=boto3.client('rekognition')
#with open('.\.test\\test_image.png', 'rb') as image:
bucket = 'thumbnails210514'
image = 'test_image.png'
response=client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':image}})
print(response)

InvalidS3ObjectException: An error occurred (InvalidS3ObjectException) when calling the DetectText operation: Unable to get object metadata from S3. Check object key, region and/or access permissions.

In [29]:
for label in response['Labels']:
    print(label['Name'])

Text
Label
Paper
Word
Advertisement
Scoreboard
Flyer
Brochure
Poster


In [31]:
client=boto3.client('rekognition')
with open('.\.test\\test_image_english.png', 'rb') as image:
    response_eng = client.detect_labels(Image={'Bytes' : image.read()})
print(response_eng)

{'Labels': [{'Name': 'Text', 'Confidence': 98.6380844116211, 'Instances': [], 'Parents': []}, {'Name': 'Label', 'Confidence': 86.48817443847656, 'Instances': [], 'Parents': [{'Name': 'Text'}]}, {'Name': 'Symbol', 'Confidence': 69.58475494384766, 'Instances': [], 'Parents': []}, {'Name': 'Paper', 'Confidence': 67.08329010009766, 'Instances': [], 'Parents': []}], 'LabelModelVersion': '2.0', 'ResponseMetadata': {'RequestId': 'c5830a7f-d928-48fe-bcbf-6420f2f5ff60', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Fri, 14 May 2021 07:01:13 GMT', 'x-amzn-requestid': 'c5830a7f-d928-48fe-bcbf-6420f2f5ff60', 'content-length': '356', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [32]:
for label in response_eng['Labels']:
    print(label['Name'])

Text
Label
Symbol
Paper


In [14]:
def detect_labels_local_file(photo):
    client=boto3.client('rekognition')
    '''
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
    '''
    response = client.detect_labels(Image=photo)
    print(response)

    return

In [15]:
detect_labels_local_file('https://i.ytimg.com/vi/8e7GcsDEKEE/default.jpg')

ParamValidationError: Parameter validation failed:
Invalid type for parameter Image, value: https://i.ytimg.com/vi/8e7GcsDEKEE/default.jpg, type: <class 'str'>, valid types: <class 'dict'>